In [1]:
# dependencies go here

## set up database

In [5]:
# Stacy start

In [ ]:
# Stacy end

## load data

In [2]:
# Jenna start

In [3]:
# Jenna end

In [4]:
# Katrina start

In [ ]:
# Katrina end

## transforming data

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# Stacy end